In [3]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb
import gmaps
from urllib.request import urlopen
import json
import requests
import difflib
import sys
import datetime
import google, googlesearch

# Google developer API key
sys.path.append('../')
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
data_df = pd.read_csv(os.path.join('./DCC_Data/', 'FinalDatasetAirplaneCrashes'))

In [5]:
df = data_df[(data_df["Fatalities"] >= 0 ) & (abs(data_df["Latitude"]) >= 0 )]
df = df.replace(np.nan, '', regex=True)

In [6]:
df['Aboard'] = pd.to_numeric(df['Aboard'])
df['Fatalities'] = pd.to_numeric(df['Fatalities'])

In [7]:
df['Percent Survived'] = df.apply(lambda x: 100 if x['Fatalities'] ==0 else x['Fatalities']/x['Aboard']*100, axis =1)
df_test =df.iloc[0:5]

In [8]:
def nearest(row):#https://stackoverflow.com/questions/20169467/how-to-convert-from-longitude-and-latitude-to-country-or-city
    lat=row['Latitude']
    lon=row['Longitude']
    params = {
        "key": gkey,
        "radius": "50000",
        "location" : str(row["Latitude"]) + "," + str(row["Longitude"])
         }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    #pdb.set_trace()  
    try:
        country = response['results'][0]['vicinity'] 
    except:
        country = ''
    #print(requests.get(base_url, params=params).url)
    #print(json.dumps(response, indent=4, sort_keys=True))
    return(country) 

In [9]:
def getplace(row):#https://stackoverflow.com/questions/20169467/how-to-convert-from-longitude-and-latitude-to-country-or-city
    lat=row['Latitude']
    lon=row['Longitude']
    params = {
        "key": gkey,
        "latlng" : str(row["Latitude"]) + "," + str(row["Longitude"])
         }
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    response = requests.get(base_url, params=params).json()  
    
    try:
        components = response['results'][0]['address_components']
        country = town = None
        for c in components:
            if "country" in c['types']:
                country = c['long_name']
    except:
        #country = nearest(row)
        country = ''
    return(country)

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df['Country'] = df.apply(getplace, axis=1)
df['Nearest_Vicinity'] = df.apply(nearest, axis=1)
df.to_csv("df_with_country_and_nearest.csv")
df.head()

In [10]:
#date = list(df['Date'].values)
#operator = list(df["Operator"].values)

text_list=[]
#Also instead of \n you must use html's <br/>
for index, row in df.iterrows():
    text_list.append('Information: <br/>' + 
                     "Date: " + str(row['Date']) + "<br/>" + 
                     "Time: " + str(row['Time']) + "<br/>" +
                     "Operator: " + str(row['Operator']) + "<br/>" +
                     "Flight #: " + str(row['Flight #']) + "<br/>" +
                     "Route: " + str(row['Route']) + "<br/>" +
                     "AC Type: " + str(row['AC Type']) + "<br/>" +
                     "Registration: " + str(row['Registration']) + "<br/>" +
                     "cn/ln: " + str(row['cn/ln']) + "<br/>" +
                     "Aboard: " + str(row['Aboard']) + "<br/>" +
                     "Aboard Passengers: " + str(row['Aboard Passengers']) + "<br/>" +
                     "Aboard Crew: " + str(row['Aboard Crew']) + "<br/>" +
                     "Fatalities: " + str(row['Fatalities']) + "<br/>" +
                     "Fatalities Passengers: " + str(row['Fatalities Passengers']) + "<br/>" +
                     "Fatalities Crew: " + str(row['Fatalities Crew']) + "<br/>" +
                     "Ground: " + str(row['Ground']) + "<br/>" +
                     "Summary: " + str(row['Summary']) + "<br/>" +
                     "Location: " + str(row['Location']) + "<br/>" +
                     "Latitude: " + str(row['Latitude']) + "<br/>" +
                     "Latitude: " + str(row['Latitude']) + "<br/>" +
                     "geometry: " + str(row['geometry']) + "<br/>"
                    )


In [11]:
rating = df["Fatalities"]
locations = df[["Latitude", "Longitude"]]

# Plot Heatmap
fig = gmaps.figure()
figure_layout = {
    'width': '2000px',
    'height': '2000px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations.dropna()[0:1000], 
                                 weights=rating.dropna()[0:1000], 
                                 dissipating=True, 
                                 max_intensity=df['Fatalities'].max()/2,
                                 point_radius=15,
                                 opacity=0.95)
# Add layer
fig.add_layer(heat_layer)
#markers = gmaps.marker_layer(locations.dropna()[0:5])
#info_box_template = ""
#plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]
marker_layer = gmaps.marker_layer(locations.dropna()[0:50],
                                  info_box_content=text_list[0:50])
# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
planecrash_df = pd.read_csv("./DCC_Data/df_with_country_with nearest.csv")
gdp_df = pd.read_csv("./DCC_Data/gdp_data.csv", engine='python')
#gdp_df['Year'] = gdp_df['Year'].astype('int64')
#gdp_df.astype({'Year':'int64'}).dtypes
#gdp_df['Year'] = gdp_df['Year'].astype(str)
#planecrash_df[planecrash_df['Date'].str.match('2019')];
#planecrash_df['Year'] = planecrash_df.apply(lambda x: float(x['Date'][0:4])     ,axis=1)
planecrash_df['Year'] = planecrash_df.apply(lambda x: x['Date'][0:4]     ,axis=1)
gdp_df['Year'] = gdp_df['Year'].astype(str)
#gdp_df['Year'] = gdp_df.apply(lambda x: x['Year'][0:4]     ,axis=1)
#planecrash_df['Year'] = planecrash_df['Year'].astype(str)
#planecrash_df['Date'] = pd.to_datetime(planecrash_df['Date'])
#planecrash_df['Year'] = planecrash_df['Date'].dt.strftime('%Y')
#planecrash_df['Year'] = planecrash_df['Date'].astype(np.int32)


planecrash_df['GDP'] = 0 #Create and fill new column
for i, row in planecrash_df.iterrows():
    try:
        planecrash_df.loc[i,'GDP'] = gdp_df[(gdp_df["Country"]==row["Country"])  & (gdp_df["Year"]==row["Year"])  ].values[0][-1]

    except:
        planecrash_df.loc[i,'GDP'] = 0
planecrash_df.tail(50)

,Unnamed: 0,Unnamed: 0.1,Date,Time,Operator,Flight #,Route,AC Type,Registration,cn/ln,...,Ground,Summary,Location,Latitude,Longitude,geometry,Percent Survived,Country,Year,GDP
4905,4917,4917,2016-07-30,07:42,Heart of Texas Hot Air Balloon Rides,NaN,Sightseeing Fentress Airpark,Kubicek BB85Z Hot Air Balloon,N2469L,NaN,...,NaN,The hot air balloon struck power lines and cra...,"Lockhart, Texas",29.883210,-97.673629,POINT (29.8832105 -97.67362920000001),100.000000,United States,2016,1.784427e+13
4906,4918,4918,2016-08-05,12:38,Emirates,521,"Thiruvanthapuram, India - Dubai, UAE",Boeing 777-31H,A6-EMW,32700/434,...,1.0,"After being cleared to land, the crew informed...","Dubai, United Arab Emirates",25.075010,55.188761,POINT (25.0750095 55.1887608818332),100.000000,United Arab Emirates,2016,3.927733e+11
4907,4919,4919,2016-08-31,11:00,Hageland Aviiation,3190,\tRussian Mission- Marshall,Cessna 208 Grand Caravan,\tN752RV,208B5088,...,0.0,A midair collision occurred between a Cessna 2...,"Russian Mission, Alaksa",61.786309,-161.364738,POINT (61.786309 -161.364738),100.000000,United States,2016,1.784427e+13
4908,4920,4920,2016-10-24,07:20,AE Aviation,NaN,Malta-Luqa - Malta-Luga,Swearingen SA277-AT Expditer,N577MX,AT-577,...,0.0,"Shortly after taking off, the plane banked to ...",Malta International Airport,35.847393,14.492267,POINT (35.84739320000001 14.4922673564904),100.000000,Malta,2016,1.372503e+10
4909,4921,4921,2016-11-28,22:00,LAMIA Bolivia,2933,"Santa Cruz, Bolivia - Medellin,Colombia",Avro RJ-85,CP-2933,E-2348,...,0.0,The crew of the chartered passenger jet declar...,"La Union, Colombia",5.951616,-75.363597,POINT (5.9516158 -75.3635969971228),92.207792,Colombia,2016,3.822160e+11
4910,4922,4922,2016-12-03,11:22,Indonesian Police,NaN,Pangkal Pinang - Batam Island,M28 Skytruck,P-4201,NaN,...,0.0,The Indonesian police aircraft crashed in the ...,"Tanjung Pinanga, Indonesia",4.640500,96.840700,POINT (4.6405 96.8407),100.000000,Indonesia,2016,1.146845e+12
4911,4923,4923,2016-12-07,16:20,Pakistan International Airlines,661,Chitral - Islamabad,ATR-42-500,AP-BHO,663,...,0.0,"While en route, the crew made a mayday call re...","Havlien, Pakistan",49.974794,14.386725,POINT (49.97479420000001 14.386725),100.000000,Czechia,2016,0.000000e+00
4912,4924,4924,2016-12-18,06:10,Indonesian Air Force,NaN,Timika - Warmera,\tLockheed C-130H Hercules,\tA-1134,4785,...,0.0,The military plane crashed into mountainous te...,"Wamena, Papua, Indonesia",-4.086235,138.937701,POINT (-4.08623515 138.937700912411),100.000000,Indonesia,2016,1.146845e+12
4913,4925,4925,2016-12-20,17:23,Aerosucre Colombia,NaN,Puerto Carreno - Bogota,Boeing 727-2JOF,HK-4544,21105/1158,...,0.0,"After taking off on runway 24, the cargo jet f...","Puerto Carreno, Colombia",6.187971,-67.489467,POINT (6.1879711 -67.48946670000001),83.333333,Colombia,2016,3.822160e+11
4914,4926,4926,2016-12-25,05:27,Russian Air Force,NaN,Chkalovsky AB -Ader - Latakia,Tupolev 154B-2,RA-85572,83A-572,...,0.0,"The plane, carrying the Alexandrov Emsemble ch...","Adar, Russia",44.976470,41.147632,POINT (44.9764703 41.14763185),100.000000,Russia,2016,0.000000e+00


In [26]:
planecrash_df = planecrash_df.dropna(subset=['Operator','Route'])
planecrash_df = planecrash_df[~planecrash_df['Operator'].str.contains('Military',case=False)]
planecrash_df = planecrash_df[planecrash_df['Route'].str.contains(' - ',case=False)]

planecrash_df.head(50)

,Unnamed: 0,Unnamed: 0.1,Date,Time,Operator,Flight #,Route,AC Type,Registration,cn/ln,...,Ground,Summary,Location,Latitude,Longitude,geometry,Percent Survived,Country,Year,GDP
32,32,32,1919-12-11,NaN,Aircraft Travel Transport,NaN,Paris - Hounslow,de Havilland DH-4,G-EAHF,NaN,...,0.0,Crashed in a field while attemptting to land.,"Catherham, Surrey, UK",51.253434,-0.474562,POINT (51.25343400000001 -0.474562),100.000000,United Kingdom,1919,0.0
34,34,34,1920-03-30,NaN,US Aerial Mail Service,NaN,Washington - Newark,De Havilland DH-4,72,NaN,...,0.0,George Sherlock was killed when his mail plane...,"Newark, New Jersey",40.735657,-74.172367,POINT (40.735657 -74.1723667),100.000000,United States,1920,0.0
49,49,49,1920-12-14,12:15,Handley Page Transport,NaN,London - Paris,Handley Page HP-16,G-EAMA,HP-25,...,0.0,The plane failed to gain altitude after taking...,"London, England",51.507322,-0.127647,POINT (51.5073219 -0.1276474),50.000000,United Kingdom,1920,0.0
61,61,61,1921-09-06,NaN,Franco-Roumaine,NaN,Warsaw - Prague - Strasbourg - Paris,Potez IX,F-ADCD,160,...,0.0,Crashed while making an approach to Le Bourget...,"Paris, France",48.856697,2.351462,POINT (48.8566969 2.3514616),100.000000,France,1921,0.0
62,62,62,1921-12-05,NaN,West Australian Airways,NaN,Geraldton - Derby,Bristol 28 Tourer,G-AUDI,6116,...,0.0,On approach to Geraldton the plane crashed nea...,"Murchinson River, Australia",-36.615389,145.238244,POINT (-36.6153889 145.2382442),100.000000,Australia,1921,0.0
65,65,65,1922-03-22,11:40,Aero Limited,NaN,Miami - Bimini,Aeromarine Model 85 (flying boat),NaN,NaN,...,0.0,The plane ditched into the ocean about 40 mile...,North Atlantic Ocean,40.000000,-40.000000,POINT (40 -40),83.333333,NaN,1922,0.0
66,66,66,1922-04-07,13:15,Daimler Airways / Grands Express Aeriens,NaN,Croydon - Le Bourget,de Havilland DH-18 / Farman F-60 Goliath,G-EAOW/FGEAD,4,...,0.0,A midair collision occurred at 400 ft. in poor...,"Grandvilliers, France",49.664777,1.942828,POINT (49.6647765 1.9428278),100.000000,France,1922,0.0
68,68,68,1922-06-03,NaN,Cie des Messageries Aeriennes,NaN,Croydon - Paris,Bleriot Spad 27,F-ACMH,555,...,0.0,Went down into the English Channel while en ro...,"Folkestone, Kent, England",51.079134,1.179407,POINT (51.07913350000001 1.1794073),100.000000,United Kingdom,1922,0.0
70,70,70,1922-07-16,16:45,Compagnie Franco-Roumaine de Navigaation Aerienne,NaN,Strasbourg - Paris,Potez 29,F-FREI,25,...,0.0,Lost an engine and could not make an emergency...,"Saverne, Lorraine, France",48.741991,7.362595,POINT (48.7419909 7.3625953),100.000000,France,1922,0.0
73,73,73,1922-10-02,NaN,de Havilland Air Service,NaN,Venice - London,de Havilland DH-9,G-EAYT,14,...,0.0,Shortly after taking off the plane crashed int...,"Venice, Italy",45.437191,12.334590,POINT (45.4371908 12.3345898),100.000000,Italy,1922,0.0


In [27]:
planecrash_df['Departure'] = planecrash_df.apply(lambda x: x['Route'].split(' - ')[0], axis=1)
planecrash_df.head(10);

,Unnamed: 0,Unnamed: 0.1,Date,Time,Operator,Flight #,Route,AC Type,Registration,cn/ln,...,Summary,Location,Latitude,Longitude,geometry,Percent Survived,Country,Year,GDP,Departure
32,32,32,1919-12-11,NaN,Aircraft Travel Transport,NaN,Paris - Hounslow,de Havilland DH-4,G-EAHF,NaN,...,Crashed in a field while attemptting to land.,"Catherham, Surrey, UK",51.253434,-0.474562,POINT (51.25343400000001 -0.474562),100.000000,United Kingdom,1919,0.0,Paris
34,34,34,1920-03-30,NaN,US Aerial Mail Service,NaN,Washington - Newark,De Havilland DH-4,72,NaN,...,George Sherlock was killed when his mail plane...,"Newark, New Jersey",40.735657,-74.172367,POINT (40.735657 -74.1723667),100.000000,United States,1920,0.0,Washington
49,49,49,1920-12-14,12:15,Handley Page Transport,NaN,London - Paris,Handley Page HP-16,G-EAMA,HP-25,...,The plane failed to gain altitude after taking...,"London, England",51.507322,-0.127647,POINT (51.5073219 -0.1276474),50.000000,United Kingdom,1920,0.0,London
61,61,61,1921-09-06,NaN,Franco-Roumaine,NaN,Warsaw - Prague - Strasbourg - Paris,Potez IX,F-ADCD,160,...,Crashed while making an approach to Le Bourget...,"Paris, France",48.856697,2.351462,POINT (48.8566969 2.3514616),100.000000,France,1921,0.0,Warsaw
62,62,62,1921-12-05,NaN,West Australian Airways,NaN,Geraldton - Derby,Bristol 28 Tourer,G-AUDI,6116,...,On approach to Geraldton the plane crashed nea...,"Murchinson River, Australia",-36.615389,145.238244,POINT (-36.6153889 145.2382442),100.000000,Australia,1921,0.0,Geraldton
65,65,65,1922-03-22,11:40,Aero Limited,NaN,Miami - Bimini,Aeromarine Model 85 (flying boat),NaN,NaN,...,The plane ditched into the ocean about 40 mile...,North Atlantic Ocean,40.000000,-40.000000,POINT (40 -40),83.333333,NaN,1922,0.0,Miami
66,66,66,1922-04-07,13:15,Daimler Airways / Grands Express Aeriens,NaN,Croydon - Le Bourget,de Havilland DH-18 / Farman F-60 Goliath,G-EAOW/FGEAD,4,...,A midair collision occurred at 400 ft. in poor...,"Grandvilliers, France",49.664777,1.942828,POINT (49.6647765 1.9428278),100.000000,France,1922,0.0,Croydon
68,68,68,1922-06-03,NaN,Cie des Messageries Aeriennes,NaN,Croydon - Paris,Bleriot Spad 27,F-ACMH,555,...,Went down into the English Channel while en ro...,"Folkestone, Kent, England",51.079134,1.179407,POINT (51.07913350000001 1.1794073),100.000000,United Kingdom,1922,0.0,Croydon
70,70,70,1922-07-16,16:45,Compagnie Franco-Roumaine de Navigaation Aerienne,NaN,Strasbourg - Paris,Potez 29,F-FREI,25,...,Lost an engine and could not make an emergency...,"Saverne, Lorraine, France",48.741991,7.362595,POINT (48.7419909 7.3625953),100.000000,France,1922,0.0,Strasbourg
73,73,73,1922-10-02,NaN,de Havilland Air Service,NaN,Venice - London,de Havilland DH-9,G-EAYT,14,...,Shortly after taking off the plane crashed int...,"Venice, Italy",45.437191,12.334590,POINT (45.4371908 12.3345898),100.000000,Italy,1922,0.0,Venice


In [28]:
def city(row):#https://stackoverflow.com/questions/20169467/how-to-convert-from-longitude-and-latitude-to-country-or-city
    params = {
        "key": gkey,
        "input": row['Departure'],
        "inputtype":"textquery"
         }
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
    response = requests.get(base_url, params=params).json()
    #print(requests.get(base_url, params=params).url) 
    try:
        place_id = response['candidates'][0]['place_id']
        #https://developers.google.com/places/web-service/details 
    except:
        place_id = ''
        
    params = {
        "key": gkey,
        "place_id": place_id
         }
    
    base_url = "https://maps.googleapis.com/maps/api/place/details/json?"
    response = requests.get(base_url, params=params).json()
    #print(requests.get(base_url, params=params).url)
    try:
        lat = response['result']['geometry']['location']['lat']
        lng = response['result']['geometry']['location']['lng']
    except:
        lat = ''
        lng = ''
    return(lat,lng) 

In [29]:
test_df = planecrash_df.head()

In [30]:
planecrash_df['Departure Lat,Lng'] = planecrash_df.apply(city, axis=1)

In [31]:
#planecrash_df.to_csv('./df_departures.csv')


In [71]:
df = pd.read_csv('./df_departures.csv')

In [73]:
#df = df.replace(',','NA', regex=True)
df = df[~df['Departure Lat,Lng'].str.contains("('', '')")]
df.head(60)

/Users/davidcoy/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Time,Operator,Flight #,Route,AC Type,Registration,...,Location,Latitude,Longitude,geometry,Percent Survived,Country,Year,GDP,Departure,"Departure Lat,Lng"
0,32,32,32,1919-12-11,NaN,Aircraft Travel Transport,NaN,Paris - Hounslow,de Havilland DH-4,G-EAHF,...,"Catherham, Surrey, UK",51.253434,-0.474562,POINT (51.25343400000001 -0.474562),100.000000,United Kingdom,1919,0.0,Paris,"(48.856614, 2.3522219)"
1,34,34,34,1920-03-30,NaN,US Aerial Mail Service,NaN,Washington - Newark,De Havilland DH-4,72,...,"Newark, New Jersey",40.735657,-74.172367,POINT (40.735657 -74.1723667),100.000000,United States,1920,0.0,Washington,"(47.7510741, -120.7401386)"
2,49,49,49,1920-12-14,12:15,Handley Page Transport,NaN,London - Paris,Handley Page HP-16,G-EAMA,...,"London, England",51.507322,-0.127647,POINT (51.5073219 -0.1276474),50.000000,United Kingdom,1920,0.0,London,"(51.5073509, -0.1277583)"
3,61,61,61,1921-09-06,NaN,Franco-Roumaine,NaN,Warsaw - Prague - Strasbourg - Paris,Potez IX,F-ADCD,...,"Paris, France",48.856697,2.351462,POINT (48.8566969 2.3514616),100.000000,France,1921,0.0,Warsaw,"(52.2296756, 21.0122287)"
4,62,62,62,1921-12-05,NaN,West Australian Airways,NaN,Geraldton - Derby,Bristol 28 Tourer,G-AUDI,...,"Murchinson River, Australia",-36.615389,145.238244,POINT (-36.6153889 145.2382442),100.000000,Australia,1921,0.0,Geraldton,"(-28.7796535, 114.6144452)"
5,65,65,65,1922-03-22,11:40,Aero Limited,NaN,Miami - Bimini,Aeromarine Model 85 (flying boat),NaN,...,North Atlantic Ocean,40.000000,-40.000000,POINT (40 -40),83.333333,NaN,1922,0.0,Miami,"(25.7616798, -80.1917902)"
6,66,66,66,1922-04-07,13:15,Daimler Airways / Grands Express Aeriens,NaN,Croydon - Le Bourget,de Havilland DH-18 / Farman F-60 Goliath,G-EAOW/FGEAD,...,"Grandvilliers, France",49.664777,1.942828,POINT (49.6647765 1.9428278),100.000000,France,1922,0.0,Croydon,"(51.376165, -0.098234)"
7,68,68,68,1922-06-03,NaN,Cie des Messageries Aeriennes,NaN,Croydon - Paris,Bleriot Spad 27,F-ACMH,...,"Folkestone, Kent, England",51.079134,1.179407,POINT (51.07913350000001 1.1794073),100.000000,United Kingdom,1922,0.0,Croydon,"(51.376165, -0.098234)"
8,70,70,70,1922-07-16,16:45,Compagnie Franco-Roumaine de Navigaation Aerienne,NaN,Strasbourg - Paris,Potez 29,F-FREI,...,"Saverne, Lorraine, France",48.741991,7.362595,POINT (48.7419909 7.3625953),100.000000,France,1922,0.0,Strasbourg,"(48.5734053, 7.752111299999999)"
9,73,73,73,1922-10-02,NaN,de Havilland Air Service,NaN,Venice - London,de Havilland DH-9,G-EAYT,...,"Venice, Italy",45.437191,12.334590,POINT (45.4371908 12.3345898),100.000000,Italy,1922,0.0,Venice,"(45.4408474, 12.3155151)"


In [75]:
Lines=[]
for i, row in df.iterrows():
    #pdb.set_trace()
    lat, lng = row["Departure Lat,Lng"].replace('(', '').replace(')', '').split(",")
    #print(lat)
    start_lat =float(lat)
    start_lng =float(lng)
    crash_lat = row['Latitude']
    crash_lng = row['Longitude']
    line = gmaps.Line((start_lat, start_lng), (crash_lat, crash_lng),stroke_weight=3.0 )
    Lines.append(line)

In [89]:
text_list=[]
#Also instead of \n you must use html's <br/>
for index, row in df.iterrows():
    text_list.append('Information: <br/>' + 
                     "Date: " + str(row['Date']) + "<br/>" + 
                     "Time: " + str(row['Time']) + "<br/>" +
                     "Operator: " + str(row['Operator']) + "<br/>" +
                     "Flight #: " + str(row['Flight #']) + "<br/>" +
                     "Route: " + str(row['Route']) + "<br/>" +
                     "AC Type: " + str(row['AC Type']) + "<br/>" +
                     "Registration: " + str(row['Registration']) + "<br/>" +
                     "cn/ln: " + str(row['cn/ln']) + "<br/>" +
                     "Aboard: " + str(row['Aboard']) + "<br/>" +
                     "Aboard Passengers: " + str(row['Aboard Passengers']) + "<br/>" +
                     "Aboard Crew: " + str(row['Aboard Crew']) + "<br/>" +
                     "Fatalities: " + str(row['Fatalities']) + "<br/>" +
                     "Fatalities Passengers: " + str(row['Fatalities Passengers']) + "<br/>" +
                     "Fatalities Crew: " + str(row['Fatalities Crew']) + "<br/>" +
                     "Ground: " + str(row['Ground']) + "<br/>" +
                     "Summary: " + str(row['Summary']) + "<br/>" +
                     "Location: " + str(row['Location']) + "<br/>" +
                     "Latitude: " + str(row['Latitude']) + "<br/>" +
                     "Longitude: " + str(row['Longitude']) + "<br/>" +
                     "geometry: " + str(row['geometry']) + "<br/>" +
                     "Departure Lat,Lng" + str(row['Departure Lat,Lng']) + "<br/>" 
                    )

In [91]:
rating = df["Fatalities"]
locations = df[["Latitude", "Longitude"]]

# Plot Heatmap
fig2 = gmaps.figure()
figure_layout2 = {
    'width': '2000px',
    'height': '2000px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(layout=figure_layout2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations.values, 
                                 weights=rating.dropna(), 
                                 dissipating=True, 
                                 max_intensity=df['Fatalities'].max()/20,
                                 point_radius=15,
                                 opacity=0.95)
# Add layer
fig2.add_layer(heat_layer)

marker_layer = gmaps.marker_layer(locations.dropna(),
                                  info_box_content=text_list)
# Display figure
fig2.add_layer(marker_layer)

# Features to draw on the map
drawing = gmaps.drawing_layer(features=Lines)
marker_layer2 = gmaps.marker_layer(locations.dropna(),
                                  info_box_content=text_list)
# Display figure
fig2.add_layer(marker_layer2)




fig2.add_layer(drawing)



fig2

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/davidcoy/Downloads/My First Project-d821d4ac1a16.json"
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS']) 

In [ ]:
import bq_helper
from bq_helper import BigQueryHelper #https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package

In [ ]:
#https://www.kaggle.com/jessicali9530/how-to-query-world-development-indicators-data
wdi = bq_helper.BigQueryHelper(active_project="patents-public-data",
                                   dataset_name="worldbank_wdi")

In [ ]:
# View table names under the worldbank_wdi data table
bq_assistant = BigQueryHelper("patents-public-data", "worldbank_wdi")
bq_assistant.list_tables()

In [ ]:
# View the first three rows of the wdi_2016 data table
df = bq_assistant.head("wdi_2016", num_rows=100)
df

In [ ]:
query1 = """
SELECT
  country_name, indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
ORDER BY 
  indicator_value DESC
LIMIT
  20;
        """
response1 = wdi.query_to_pandas_safe(query1)
response1.head(20)